# Trajectory optimization

As a robot moves around the world, it can accumulates readings of its own motion. _"I have moved a meters forward!"_, a robot may exclaim after querying a sensor. _"One meter forward and twenty to the left!"_, after another reading. And you get how that goes.

After some time, a robot may have collected a _sequence_ of measurements that, when put together, are representative of its entire trajectory. However, there is an immediate danger in this formulation! If a single measurement is wrong, the rest will carry over its error.

For example, if we have the readings:
* `forward 1m and rotate 90 degree to the left`, 
* `forward 1m and rotate 90 degree to the left`, 
* `forward 1m and rotate 90 degree to the left`, 
* `forward 1m and rotate 90 degree to the left`, 

the robot believes that it drew a square and got back to its starting position. 

However, if the readings change to 

* `forward 1m and rotate 90 degree to the left`, 
* `forward 1m and rotate 90 degree to the left`, 
* `forward 1m and rotate` **45 degree to the left**, 
* `forward 1m and rotate 90 degree to the left`, 

then the robot believes that it traveled to a point to the left of its starting point. This is called odometry drift. It is the scenario where error in odometry readings get carried over to the next position estimates.

Hope is not all lost, however! Given additional information, robots can correct the estimates of their position in the world and also correct backwards (smooth) their entire trajectory. 
Back to our simple example, if the robot knew somehow that the end position of its third motion was one meter to the left of the origin, then it would get there corrected readings:

* `forward 1m and rotate 90 degree to the left`, 
* `forward 1m and rotate 90 degree to the left`, 
* ~~`forward 1m and rotate` 45 degree to the left~~  **at one meter to the left of the origin and facing it** 
* `forward 1m and rotate 90 degree to the left`, 

and the robot would believe that it is back at the initial position and also that it has turned 90 degrees in its third motion.

In this section we'll be exploring this problem, which we call **trajectory optimization**, in depth.

We'll be 
* Discussing rigid transformations in 2-dimensions.
* Discussing landmark observations.
* Discussing optimization problems in general.
* Formalizing the trajectory optimization problem.
* Solving the optimization problem with GTSAM.

Let's get to work!

# TODO add figure for odometry drift.

# Rigid Transformations in 2D

To converse about motion and localization in robotics it is helpful to agree on a couple of common conventions.

## **Directions**

We'll adopt the right-handed coordinate system for our discussion.
* The front of the robot faces the positive $x$ direction.
* The left side of the robot faces the positive $y$ direction.
* The top of the robot faces the positive $z$ direction. Since we operate in two dimensions, we care less for this axis right now.
* A positive rotation angle $\theta$ is according to the right-hand-rule, with the thumb pointing towards the $z$ direction and zero being on the $x$ axis.

## **Poses** 

We will parameterize a trajectory as a sequence of poses, $p_0, p_1, p_2, \dots, p_N$. Each pose is defined as a position (also called translation) and a rotation (also called orientation) in a fixed "world" frame. This world frame could be the initial pose of the robot, its charging station, some well known corner of a room, or another arbitrary position and orientation.

Note that we have specified a point  $(x,y)$ with a rotation $\theta$, within some world frame $w$. This means that the pose $(x=1,y=0,\theta=10)$ lies exactly one meter ahead of the world origin and rotated from its $x$ axis by 10 degrees to the left (it is almost aligned with its $x$ axis), for example. It is important to be complete about our notation to avoid confusion! So let's call this transformation $X^w_0$, denoting the transformation **from** the world frame $w$ **to** the zero-th pose $p_0$.
In this case we call $w$ the parent frame and $0$ the child frame.

To build some intuition, what would be the transformation $X_w^0$?

It would be the origin of the world frame $w$ in the frame of $op_0$. The transformation **from** $p_0$ to $w$. 

## **Transformations**

So far, we have talked about poses in a world frame. We can continue our discussion and also talk about transformations between any frame to any other frame! As an example, we could ask where is the robot one second after the last time we have checked? Or in other words, _what is the transformation between time step $i$ to time step $i+1$?_ In general terms, it would be an object that denotes the motion of the robot. If we were standing at the pose of the robot at time $t$, then our transformation would tell us how to move to reflect the robot's pose at time $t+1$. We'll denote this kind of object as $X^i_{i+1}$. 

A transformation in two dimensions is a member of the "special euclidean" ($SE(2)$) group. This group is a group of matrices that encode rotations and translations. Each matrix $X \in SO(2)$ (_"$X$ in $SE(2)$"_) is composed of a rotation matrix $R$ in the "special orthogonal" ($SO(2)$) group and a translation vector $\mathbf{t}\in\mathbb{R}^2$. The rotation matrix is allowed to, obviously perhaps, rotate poses in space but not to scale or shear them. (Formally, it has to be orthogonal and have determinant with value 1.) This makes sense as a rotating robot often cannot change its shape. The translation vector takes on the shape of $[\Delta x, \Delta y]^\top$, and encodes the motion of a robot along its $x$ and $y$ axes. The constraint on $\mathbf{t}$ is such that its components are real numbers. 

If a robot moved by some $\Delta x, \Delta y$ meters and turned by $\Delta \theta$ radians, then we can ask ourselves a couple of questions:

### Where would the robot origin be after the motion, with respect to its initial position?

This is a simple case! We mark the robot's origin at time $i$ as the origin, and can see that the motion is exactly the transformation between the origin (robot position at time $i$) and its pose at time $i+1$ with respect to that initial position. Thus, the robot pose at time $i+1$ in the frame $i$, which is denoted with the $i$ superscripts and the $i+1$ subscripts, is the following.


$$\theta_{i+1}^i = \Delta \theta$$
$$\mathbf{t}^{i}_{i+1} = \begin{bmatrix}
\Delta x\\
\Delta y\\
\end{bmatrix}$$

In our shorthand notation, we can write the rotation matrix and translation vector

$$R_{i+1}^i = \begin{bmatrix}
\cos(\Delta \theta) & \sin(\Delta \theta)\\
-\sin(\Delta \theta) & \cos(\Delta \theta)\\
\end{bmatrix}, \ \ \ 
\mathbf{t}_{i+1}^i = \begin{bmatrix}
\Delta x\\
\Delta y\\
\end{bmatrix}$$

$$ X^i_{i+1} = \begin{bmatrix}
R^i_{i+1} & \mathbf{t}^i_{i+1} \\
\mathbf{0} &1
\end{bmatrix}$$

### Where would any other point on the robot be, with respect to the robot's initial position, after the motion?

When looking at a point away from the origin of the robot, like a position of a camera with respect to the robot origin that we denote $X^b_c$ ("camera in the base frame"), its new positions in the robot frame from time $i$ after the motion will not simply be the rotation and translation as specified by the motion. To compute the new position of the point on the robot, we'll first rotate it to the final robot configuration with the matrix multiplication $R^i_{i+1}X^b_c$ and then translate it with the robot translation $\mathbf{t}^i_{i+1}$. This can be written as 


$$\begin{bmatrix}
{x^b_c}_{, i+1}\\
{y^b_c}_{, i+1}
\end{bmatrix} = R^{i}_{i+1} \begin{bmatrix}
{x^b_c}_{, i}\\
{y^b_c}_{, i}
\end{bmatrix} + \mathbf{t}^i_{i+1}$$


And in our shorthand:

$$\begin{bmatrix}
{x^b_c}_{, i+1}\\
{y^b_c}_{, i+1}\\
1
\end{bmatrix} = X^{i}_{i+1} \begin{bmatrix}
{x^b_c}_{, i}\\
{y^b_c}_{, i}\\
1
\end{bmatrix}$$


### Where would the origin of the robot be, with respect to the world frame, after the motion that is specified in the robot frame.

To answer question about the robot pose in the world frame, we first need to have information about its initial position in the world frame $X^w_i$. Let's assume that we have this information. As earlier, we also have information about the robot motion in its ows frame, which is $X^i_{i+1}$. We are interested in finding out the robot pose at time $i+1$, in the world frame, after its motion. This tranformation is denoted as $X^w_{i+1}$. I would argue that we have all the information that we need!

For starters, we can find the orientation of the robot. The rotations simply add!

$$\theta_{i+1}^w = \theta_i^w + \Delta \theta$$

As for the translation, we are going to do an operation that may not seem trivial at first, but I believe that if you stare at it for a bit you'll see that it is logical.

We'll "transform" the robot motion, that has been specified in its own frame, to "begin" at its pose as specified in the world frame.
This is written as:

$$X^w_{i+1} = X^w_{i} X^{i}_{i+1}$$

Note how the subscripts and superscripts of the frames of reference match in a zig-zag fashion. This notation is very useful and reduces confusion! It has been adapted from the MIT Robotic Manipulation class https://manipulation.csail.mit.edu/.

### Chaining Transformations.
A cool thing that directly follows from the computation of $X^w_{i+1}$, is that we notice how we can add together a bunch of motions from a robot trajectory to compute a final robot pose. 

Say we are given a set of motions steps that occurred at times $i, i+1, i+2, i+3$, and also the initial robot configuration at time $i$ (call it $[x^w_i, y^w_i, \theta^w_i]$ or $[X^w_i, \theta^w_i]$). The we can compute the final robot orientation and translation as

$$\theta_{i+3}^w = \theta_i^w + \Delta \theta^{i}_{i+1} + \Delta \theta^{i+1}_{i+2} + \Delta \theta^{i+2}_{i+3}$$

$$X^w_{i+3} = X^w_{i} X^{i}_{i+1} X^{i+1}_{i+2} X^{i+2}_{i+3}$$

Simple! Right? This is the way we can compute the robot pose after it moves in space, and all the information we have is its motion steps and initial position.


<!-- $$ X^w_i = X^{i}_{i+1}X^w_{i+1}$$ -->


### Inverse transformations

Finally, we can also write down the _inverse_ transformation between two frames give a transformation between them. Specifically, if we have $X^i_{i+1}$, we'd like to know what is $X^{i+1}_i$, or "if the robot would start at time $i+1$, how should it move to get to time $i$ (with respect to time $i+1$ as origin)?".
This operation will be useful later.

$$\theta_i^{i+1} = -\theta_{i+1}^i$$

$$ X^{i+1}_i = \left( X^{i+1}_i \right)^{-1} =  \begin{bmatrix}
(R^i_{i+1})^\top & -(R^i_{i+1})^\top \mathbf{t}^i_{i+1} \\
\mathbf{0} &1
\end{bmatrix}$$



# The Trajectory Python Class

Phew! That was a lot of text. Let's shift gears and start implementing some code!

Our code in this project will all be packaged withing the `Trajectory` class.

We will use this class to keep track of robot trajectories and also optimize those when we have multiple measurements. What would we want to implement then? What are our inputs?

Our inputs are
1. A sequence of robot motions $[\Delta x, \Delta y, \Delta \theta]$.
2. Observations of landmarks. We'll discuss those soon.

Our outputs are:
1. Optimized trajectory. 

### Implementing a transformation from human-readable inputs.

In [1]:
import numpy as np
import gtsam

In [ ]:
class Trajectory():
    def __init__(self) -> None:
        # Noise.
        self.ODOM_NOISE =     gtsam.noiseModel.Diagonal.Sigmas(np.array([0.3, 0.3, 0.1]))
        self.LANDMARK_NOISE = gtsam.noiseModel.Diagonal.Sigmas(np.array([0.03, 0.03, 0.03]))
        self.PRIOR_NOISE =    gtsam.noiseModel.Diagonal.Sigmas(np.array([0.00003, 0.00003, 0.00001]))

        # Landmarks.
        self.landmarks = set()

        # Measurements. Mapping nodes to RangeMeasurement's/
        self.measurements = {}
        
        # Keep track of xy edges and build matrix C only after optimizing for angles.
        self.edges = []

        # Should we optimize after every edge addition?
        self.online = False

### Inverse Transformations.

# Landmark observations.
What and why

Observation model. Adding observations to our class.
Relating pose estimates via landmark observations (loop closures)

### Creating virtual measurements between poses.

In [ ]:
# Given a dictionary relating odometry pose estimates to landmarks, and a relative pose between an odometry estimate and some landmark return a list of relative poses between the other nodes that have seen this landmark.

### Optimization Primer

In this section we are not going to dive too deep into the background of optimization theory. In fact, we will almost completely leave the actual optimization of the pose graph  (the trajectory optimization) as a black box.

It is helpful, however, to understand what optimization is and how it could be solved.

In [ ]:
# Least squares line fitting.


### Trajectory Optimization.

Formulation of cost function and analogy to least squares linear regression

provide code for optimization.